In [ ]:
import sys
sys.path.append('C:/Users/Hendrik/PycharmProjects') # go to parent dir

from ActiveCritic.model_src.whole_sequence_model import WholeSequenceActor, WholeSequenceCritic, WholeSequenceModelSetup
from ActiveCritic.model_src.transformer import TransformerModel, CriticTransformer, ModelSetup
import torch as th
from ActiveCritic.tests.test_utils.utils import make_mask_data, make_seq_encoding_data, make_critic_data, make_wsm_setup

In [ ]:
import torch

In [ ]:
wsm = WholeSequenceModelSetup()
wsm.model_setup = ModelSetup()
seq_len = 6
ntoken = 3
d_output = 2
batch_size = 2
d_intput = 3
wsm.model_setup.d_output = d_output
wsm.model_setup.nhead = 1
wsm.model_setup.d_hid = 10
wsm.model_setup.d_model = 10
wsm.model_setup.nlayers = 2
wsm.model_setup.seq_len = seq_len
wsm.model_setup.dropout = 0
wsm.lr = 1e-3
wsm.model_setup.device = 'cuda'
wsm.optimizer_class = th.optim.Adam
wsm.optimizer_kwargs = {}
wsm.model_setup.model_class:TransformerModel = TransformerModel

In [ ]:
wsm = make_wsm_setup(seq_len=seq_len, d_output=d_output, model_class=TransformerModel)

In [ ]:
wsa = WholeSequenceActor(wsms=wsm)

In [ ]:
input = th.ones([batch_size, seq_len, d_intput], dtype=th.float, device='cuda')

In [ ]:
output = wsa.forward(inputs=input)

In [ ]:
shape = output.shape
assert shape[0] == batch_size
assert shape[1] == seq_len
assert shape[2] == d_output

In [ ]:
inpt_seq, outpt_seq = make_seq_encoding_data(batch_size=batch_size, seq_len=seq_len, ntoken=ntoken, d_out=d_output)
success = th.ones_like(inpt_seq, dtype=th.bool)

In [ ]:
wsa = WholeSequenceActor(wsms=wsm)
data = inpt_seq, outpt_seq, success
for i in range(3000):
    result = wsa.forward(inputs=inpt_seq)
    resp = wsa.optimizer_step(data=data)

In [ ]:
res = wsa.optimizer_step(data=data)
assert res['Trajectory Loss '] < 1e-2

res = wsa.optimizer_step(data=data, prefix='test')
assert 'Trajectory Loss test' in res


In [ ]:
wsa.init_model()

In [ ]:
data = inpt_seq, outpt_seq, success
for i in range(1000):
    result = wsa.forward(inputs=inpt_seq)
    wsa.optimizer_step(data=data)

In [ ]:
res = wsa.optimizer_step(data=data)
assert res['Trajectory Loss '] < 1e-2

In [ ]:
wsm = WholeSequenceModelSetup()
wsm.model_setup = ModelSetup()
seq_len = 6
ntoken = 3
d_result = 1
d_output = 2
batch_size = 2
d_intput = 3
wsm.model_setup.d_output = d_output
wsm.model_setup.nhead = 1
wsm.model_setup.d_hid = 10
wsm.model_setup.d_model = 10
wsm.model_setup.nlayers = 2
wsm.model_setup.seq_len = seq_len
wsm.model_setup.dropout = 0
wsm.lr = 1e-3
wsm.model_setup.d_result = d_result
wsm.model_setup.device = 'cuda'
wsm.optimizer_class = th.optim.Adam
wsm.optimizer_kwargs = {}
wsm.model_setup.model_class:TransformerModel = CriticTransformer

In [ ]:
wsc = make_wsm_setup(seq_len=seq_len, d_output=d_output, model_class=CriticTransformer, d_result=d_result)

In [ ]:
inpt_seq, outpt_seq = make_critic_data(batch_size=batch_size, seq_len=seq_len, ntoken=ntoken)
data = inpt_seq, None, outpt_seq
model = WholeSequenceCritic(wsc)
for i in range(3000):
    res = model.optimizer_step(data)

In [ ]:
res

In [ ]:
model.init_model()

In [ ]:
res = model.optimizer_step(data)

In [ ]:
res

In [ ]:
wsm = WholeSequenceModelSetup()
wsm.model_setup = ModelSetup()
seq_len = 6
ntoken = 3
d_output = 2
batch_size = 2
d_intput = 3
wsm.model_setup.d_output = d_output
wsm.model_setup.nhead = 1
wsm.model_setup.d_hid = 10
wsm.model_setup.d_model = 10
wsm.model_setup.nlayers = 2
wsm.model_setup.seq_len = seq_len
wsm.model_setup.dropout = 0
wsm.lr = 1e-3
wsm.model_setup.device = 'cuda'
wsm.optimizer_class = th.optim.AdamW
wsm.optimizer_kwargs = {}
wsm.model_setup.model_class:TransformerModel = TransformerModel
wsa = WholeSequenceActor(wsms=wsm)
input = th.ones([batch_size, seq_len, d_intput], dtype=th.float, device='cuda')
output = wsa.forward(inputs=input)
shape = output.shape

assert(shape[0] == batch_size)
assert(shape[1] == seq_len)
assert(shape[2] == d_output)

inpt_seq, outpt_seq = make_seq_encoding_data(batch_size=batch_size, seq_len=seq_len, ntoken=ntoken, d_out=d_output)
success = th.ones_like(inpt_seq, dtype=th.bool)
wsa = WholeSequenceActor(wsms=wsm)
data = inpt_seq, outpt_seq, success
for i in range(1000):
    res = wsa.optimizer_step(data=data)
assert(res['Trajectory Loss '] < 1e-2)

res = wsa.optimizer_step(data=data, prefix='test')
assert('Trajectory Loss test' in res)

wsa.init_model()
res = wsa.optimizer_step(data=data)
assert(res['Trajectory Loss '] > 1e-1, 'Init Model did not cange the parameters.')
for i in range(1000):
    res = wsa.optimizer_step(data=data)   
assert(res['Trajectory Loss '] < 1e-2, 'Did not converge after reinit.')

In [ ]:
res

In [1]:
from active_critic.model_src.whole_sequence_model import WholeSequenceModel
from active_critic.model_src.transformer import ModelSetup, TransformerModel
import torch as th
from active_critic.utils.test_utils import make_mask_data, make_seq_encoding_data, make_critic_data, make_wsm_setup
import unittest
from active_critic.utils.pytorch_utils import get_rew_mask, get_seq_end_mask

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [3]:
from active_critic.utils.pytorch_utils import calcMSE


th.manual_seed(0)

seq_len = 6
ntoken = 3
d_output = 2
batch_size = 2
d_intput = 3
current_step = 3

input = th.ones([batch_size, seq_len, 1],
                dtype=th.float, device='cuda')
input[0,0] = 3
input[1,1] = 2
org_input = th.clone(input)
input.requires_grad = True

goal = th.ones([batch_size, seq_len, 1],
        dtype=th.float, device='cuda')

opt = th.optim.SGD([input], lr=1e-1)

wsa_setup = make_wsm_setup(seq_len=seq_len, d_output=d_output)
wsm = WholeSequenceModel(wsms=wsa_setup)
mask = get_seq_end_mask(input, current_step)
print(input[mask])
print(goal[mask])
print(calcMSE(input[mask], goal[mask]))
loss = wsm.loss_fct(result=input, label=goal, mask=mask)

assert loss == 0

tensor([1., 1., 1., 1., 1., 1.], device='cuda:0', grad_fn=<IndexBackward0>)
tensor([1., 1., 1., 1., 1., 1.], device='cuda:0')
tensor(0., device='cuda:0', grad_fn=<MeanBackward0>)


In [23]:
th.manual_seed(0)

seq_len = 6
ntoken = 3
d_output = 2
batch_size = 2
d_intput = 3
current_step = 3

input = th.ones([batch_size, seq_len, 1],
                dtype=th.float, device='cuda')
input[0,0] = 3
input[1,1] = 2
org_input = th.clone(input)
input.requires_grad = True

goal = th.ones([batch_size, seq_len, 1],
        dtype=th.float, device='cuda')

opt = th.optim.SGD([input], lr=1e-1)

wsa_setup = make_wsm_setup(seq_len=seq_len, d_output=d_output)
wsm = WholeSequenceModel(wsms=wsa_setup)
mask = get_seq_end_mask(input, current_step)
loss = wsm.loss_fct(result=input, label=goal, mask=mask)

assert loss == 0

In [24]:
current_step = 1
mask = get_seq_end_mask(input, current_step)
loss = wsm.loss_fct(result=input, label=goal, mask=mask)
assert loss == 1/(batch_size*(seq_len-current_step))

In [27]:
goal[0,3:] = -1
mask = get_rew_mask(reward=goal)
print(mask)
loss = wsm.loss_fct(result=input, label=goal, mask=mask)
print(loss)
assert th.allclose(loss, th.tensor(5 / 9))

tensor([[ True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True]], device='cuda:0')
tensor(0.5556, device='cuda:0', grad_fn=<MeanBackward0>)


In [ ]:
def get_rew_mask(reward):
    return (reward.squeeze()>=0)

actions = th.randint(0, 10, [2,3,4])
actions[:,:1] = 0
rew = th.ones([2,3,1])
rew[:,:1] = -1
actions
mask = get_rew_mask(rew)
actions[mask]
assert th.equal(actions[mask].reshape(-1), actions[:,1:].reshape(-1))

In [ ]:
actions = th.randint(0, 10, [2,3,4])
actions[:,:1] = 0
rew = th.ones([2,3,1])
rew[:,:1] = -1
actions

In [ ]:
mask = get_rew_mask(rew)

In [ ]:
actions[mask]
assert th.equal(actions[mask].reshape(-1), actions[:,1:].reshape(-1))

In [ ]:
th.manual_seed(0)

seq_len = 6
change_spot = 4
ntoken = 3
d_output = 2
batch_size = 2
d_intput = 3
current_step = 1

input = th.zeros([batch_size, seq_len, 1],
                dtype=th.float, device='cuda')
input[:,change_spot:] = 1
org_input = th.clone(input)
input.requires_grad = True

goal = th.ones([batch_size, seq_len, 1],
        dtype=th.float, device='cuda')

opt = th.optim.SGD([input], lr=1e-1)

wsa_setup = make_wsm_setup(seq_len=seq_len, d_output=d_output)
oec = OptimizeEndCritic(wsms=wsa_setup)
loss = oec.loss_fct(result=input, label=goal, current_step=current_step)
assert (loss - (change_spot-current_step)/(seq_len-current_step)) == 0

In [ ]:
(change_spot-current_step)/(seq_len-current_step)

In [ ]:
input